In [1]:
import numpy as np
import tensorflow as tf

In [2]:
ten1 = np.array([1., 2., 3.]) # tensor with rank 1
ten1.shape

(3,)

In [3]:
ten2 = np.array([[1., 2., 3.], [4., 5., 6.]]) # tensor with rank 2
ten2.shape

(2, 3)

In [4]:
ten3 = np.array([[[1., 2., 3.]], [[7., 8., 9.]]]) # tensor with rank 3
ten3.shape

(2, 1, 3)

In [5]:
ten3[0]

array([[ 1.,  2.,  3.]])

In [6]:
ten3[0][0]

array([ 1.,  2.,  3.])

In [7]:
ten3[1]

array([[ 7.,  8.,  9.]])

The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions

A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point Tensors node1 and node2 as follows:



In [8]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)
print node1, node2

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [29]:
# to evaluate the nodes we need to run the session
sess = tf.Session()
node3 = tf.add(node1, node2)

In [30]:
print ("node3: ", node3)
print "sess.run(node3): ", sess.run(node3)

('node3: ', <tf.Tensor 'Add_2:0' shape=() dtype=float32>)
sess.run(node3):  7.0


In [31]:
# Placeholder -> promise to provide a value later
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b

In [32]:
print sess.run(adder_node, {a: 3, b:4.5})
print sess.run(adder_node, {a: [1, 3], b:[ 2, 4]})

7.5
[ 3.  7.]


In [33]:
add_and_three = adder_node * 3 # multiplies the output of adder_node by 3
print sess.run(add_and_three, {a: 3, b: 4})

21.0


In [35]:
writer = tf.summary.FileWriter("/home/daniel/study/seminar_DeepDreaming/log_simple", sess.graph)
writer.flush()

In [14]:
# Variables allow us to add trainable parameters to a graph
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [15]:
# constants are initialized when tf.constant is called, but Variables need to be explicitly initialized, as here
init = tf.global_variables_initializer() # init is just a handle, only when sess.run(init) is called, variables are initialized
sess.run(init)

In [16]:
print(sess.run(linear_model, feed_dict={x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [17]:
# loss function example
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas) # calculates sum of all values in vector Sum((o-y)^2)
print (sess.run(loss, feed_dict={x : [1, 2, 3, 4], y : [0, -1, -2, -3]}))

23.66


In [18]:
# Assigning values to variables W and b to reduce loss
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print sess.run(W)

[-1.]


In [19]:
print sess.run(fetches=loss, feed_dict={x:[1, 2, 3, 4], y:[0, -1, -2, -3]})

0.0


## Training models

In [20]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [21]:
sess.run(init) 
for i in xrange(1000):
    sess.run(train, {x : [1, 2, 3, 4], y : [0, -1, -2, -3]}) # running 1000 iterations of Gradient Descent

print sess.run([W, b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [22]:
loss = sess.run(loss, {x : [1, 2, 3, 4], y : [0, -1, -2, -3]})

In [23]:
print loss

5.69997e-11


In [24]:
# Creating custom linear model using tf.train, tf.train.contrib

def model(features, labels, mode):
    
    # build a linear model
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    # loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                     tf.assign_add(global_step, 1)) # group groups multiple operations
    
    # ModelFnOps connects subgraphs we built to the apprpriate functionality
    return tf.contrib.learn.ModelFnOps(mode = mode, predictions = y,
                                      loss = loss, train_op = train)

estimator = tf.contrib.learn.Estimator(model_fn = model)

# define dataset
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000) #inputs the numpy array

# train
estimator.fit(input_fn=input_fn, steps=1000)


print estimator.evaluate(input_fn = input_fn, steps=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb260595090>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpKdyv3Q/model.ckpt.
INFO:tensorflow:loss = 38.6002678959, step = 1
INFO:tensorflow:global_step/sec: 1419.63
INFO:tensorflow:loss = 0.101286566491, step = 101
INFO:tensorflow:global_step/sec: 1454.08
INFO:tensorflow:loss = 0.00503510042247, step = 201
INFO:tensorflow:global_step/sec: 1402.8
INFO:tensorflow:loss = 0.000559028409071, step = 

In [36]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)
print node1, node2

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [40]:
# to evaluate the nodes we need to run the session
sess2 = tf.Session()
node3 = tf.add(node1, node2)
sess2.run(node3)

7.0

In [42]:
writer = tf.summary.FileWriter("/home/daniel/study/seminar_DeepDreaming/log_simple", sess2.graph)
writer.flush()